In [1]:
#data science - standard library
import pandas as pd
import numpy as np

# visualizations
import matplotlib.pyplot as plt
import seaborn as sns

from api_keys import g_key
import gmaps

import requests
import json

In [2]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [3]:
df = pd.read_csv("cities.csv")
df.head()

,City,Latitude,Longitude,Temperature,Feels Like,Humidity,Pressure,Wind Speed,Cloudiness
0,Saskylakh,71.92,114.08,-7.91,-22.68,86,1008,14.38,13
1,Te Anau,-45.42,167.72,57.13,52.84,55,1018,4.05,92
2,Busselton,-33.65,115.33,60.37,53.62,82,996,14.65,97
3,Puerto Ayora,-0.74,-90.35,69.01,70.34,83,1012,5.99,11
4,Hirado,33.36,129.55,56.39,45.70,47,1022,13.87,20


In [4]:
# Store latitude and longitude in locations
locations = df[["Latitude", "Longitude"]]

# Fill NaN values and convert to float
humids = df["Humidity"].astype(float)
locations

,Latitude,Longitude
0,71.92,114.08
1,-45.42,167.72
2,-33.65,115.33
3,-0.74,-90.35
4,33.36,129.55
...,...,...
549,66.83,70.83
550,13.47,123.62
551,33.08,-16.33
552,-3.45,104.25


## Humidity Heatmap

In [5]:
coords = df[["Latitude", "Longitude"]]
humids = df.Humidity

# Customize the size of the figure
figure_layout = {
    'width': '1200px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout, map_type = "SATELLITE")

# # Assign the marker layer to a variable
# markers = gmaps.marker_layer(coords, info_box_content=states)
# # Add the layer to the map
# fig.add_layer(markers)

heat = gmaps.heatmap_layer(coords, weights = humids, dissipating=False, max_intensity=500)
# Add the layer to the map
fig.add_layer(heat)

fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

In [6]:
df.head()

,City,Latitude,Longitude,Temperature,Feels Like,Humidity,Pressure,Wind Speed,Cloudiness
0,Saskylakh,71.92,114.08,-7.91,-22.68,86,1008,14.38,13
1,Te Anau,-45.42,167.72,57.13,52.84,55,1018,4.05,92
2,Busselton,-33.65,115.33,60.37,53.62,82,996,14.65,97
3,Puerto Ayora,-0.74,-90.35,69.01,70.34,83,1012,5.99,11
4,Hirado,33.36,129.55,56.39,45.70,47,1022,13.87,20


## Create new DataFrame fitting weather criteria

In [7]:
mask = (df.Temperature < 77) & (df.Temperature > 74) & (df["Wind Speed"] < 5)
df_sub = df.loc[mask].reset_index(drop=True)
df_sub.head()

,City,Latitude,Longitude,Temperature,Feels Like,Humidity,Pressure,Wind Speed,Cloudiness
0,Yaan,7.38,8.57,76.57,78.75,65,1013,4.70,28
1,Bengkulu,-3.80,102.27,76.17,84.20,91,1011,2.44,99
2,Airai,-8.93,125.41,75.67,72.90,40,1011,4.90,37
3,Meulaboh,4.14,96.13,76.59,83.70,86,1010,2.86,98
4,Payo,-3.75,103.64,75.56,83.35,89,1011,1.66,94


## Hotel Map

In [8]:
df_sub.City.values

array(['Yaan', 'Bengkulu', 'Airai', 'Meulaboh', 'Payo', 'Mersing',
       'Calabar', 'Prestea', 'Lavumisa', 'Nikki', 'Caucayá', 'Amapá',
       'Alto Araguaia', 'Sur', 'Touros', 'Birao', 'Doka',
       'Nova Olinda do Norte'], dtype=object)

In [9]:
#try to plot the hotels with pin containing the Hotel Name first
coords = df[["Latitude", "Longitude"]]
coords_ideal = df_sub[["Latitude", "Longitude"]]
humids = df.Humidity

# Customize the size of the figure
figure_layout = {
    'width': '1200px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout, map_type = "SATELLITE")

# Assign the marker layer to a variable
markers = gmaps.marker_layer(coords_ideal, info_box_content=df_sub.City)
# Add the layer to the map
fig.add_layer(markers)

heat = gmaps.heatmap_layer(coords, weights = humids, dissipating=False, max_intensity=500)
# Add the layer to the map
fig.add_layer(heat)

fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

In [10]:
#inits
names = []
addresses = []
# prices = []
# ratings = []

# find the closest restaurant of each type to coordinates

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "rankby": "distance",
    "type": "hotel",
    "key": g_key,
}

# use iterrows to iterate through pandas dataframe
for index, row in df_sub.iterrows():
    
    location = f"{row.Latitude}, {row.Longitude}"

    # add keyword to params dict
    params['location'] = location

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row.City}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest Hotel is {results[0]['name']}.")
        
#         types_df.loc[index, 'name'] = results[0]['name']
#         types_df.loc[index, 'address'] = results[0]['vicinity']
#         types_df.loc[index, 'price_level'] = results[0]['price_level']
#         types_df.loc[index, 'rating'] = results[0]['rating']
              
        name = results[0]['name']
        address = results[0]['vicinity']
#         price = results[0]['price_level']
#         rating = results[0]['rating']
              
        names.append(name)
        addresses.append(address)
#         prices.append(price)
#         ratings.append(rating)
              
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        names.append(np.nan)
        addresses.append(np.nan)
#         prices.append(np.nan)
#         ratings.append(np.nan)
        
    print("------------")
              
df_sub["hotel_name"] = names
df_sub["address"] = addresses
# df_sub["price_level"] = prices
# df_sub["rating"] = ratings

Retrieving Results for Index 0: Yaan.
Closest Hotel is Ishom Nursery And Primary School.
------------
Retrieving Results for Index 1: Bengkulu.
Closest Hotel is PT. Zikri berkah Abadi.
------------
Retrieving Results for Index 2: Airai.
Closest Hotel is Raw Material Atsabe.
------------
Retrieving Results for Index 3: Meulaboh.
Closest Hotel is SMILE PONSEL.
------------
Retrieving Results for Index 4: Payo.
Closest Hotel is Priamanaya energi.
------------
Retrieving Results for Index 5: Mersing.
Closest Hotel is Kedai Emas Zhen Mei.
------------
Retrieving Results for Index 6: Calabar.
Closest Hotel is Jubilee Gospel centre.
------------
Retrieving Results for Index 7: Prestea.
Closest Hotel is Physical.
------------
Retrieving Results for Index 8: Lavumisa.
Closest Hotel is Lavumisa.
------------
Retrieving Results for Index 9: Nikki.
Closest Hotel is Stade Bio Passo - Football Nikki.
------------
Retrieving Results for Index 10: Caucayá.
Closest Hotel is CASA LUDICA.
------------
Re

In [11]:
list_info = []

for indx, row in df_sub.iterrows():
    info = f"City: {row.City}<br>-----------<br>Hotel: {row.name}<br>-----------<br>Address: {row.address}"
    list_info.append(info)

In [12]:
coords = df[["Latitude", "Longitude"]]
coords_ideal = df_sub[["Latitude", "Longitude"]]
humids = df.Humidity

# Customize the size of the figure
figure_layout = {
    'width': '1200px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout, map_type = "SATELLITE")

# Assign the marker layer to a variable
markers = gmaps.marker_layer(coords_ideal, info_box_content=list_info)
# Add the layer to the map
fig.add_layer(markers)

heat = gmaps.heatmap_layer(coords, weights = humids, dissipating=False, max_intensity=500)
# Add the layer to the map
fig.add_layer(heat)

fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…